# Load data

In [ ]:
import pandas as pd
df_cleaned = pd.read_csv("/content/drive/MyDrive/HateSpeech/TODA(CLEANED)/BERT/TODA_CLEANED_01.csv").iloc[:,1:]
df_cleaned = df_cleaned.dropna()

In [ ]:
df_typo = pd.read_csv("/content/drive/MyDrive/HateSpeech/TODA(CLEANED)/BERT/TODA_TYPO_01.csv").iloc[:,1:]
df_typo = df_typo.dropna()

In [ ]:
print(df_cleaned.shape, df_typo.shape)
pd.concat([df_typo.sample(5,random_state=2),df_cleaned.sample(5,random_state=2)]).sort_index()

(321596, 6) (321596, 7)


,text_raw,text_cleaned,typo_corrected,tokens_raw,tokens_stop,tokens_stems,class
16876,RT @unpccomix: @MT8_9 Thought you might like o...,rt thought you might like one of my comics f...,rt thought you might like one of my comics f...,"['rt', ' ', 'thought', 'you', 'might', 'like'...","['rt', 'thought', 'like', 'comics', 'unc', 'co...","['rt', 'thought', 'like', 'comic', 'unc', 'com...",1
16876,RT @unpccomix: @MT8_9 Thought you might like o...,rt thought you might like one of my comics f...,NaN,"['rt', ' ', 'thought', 'you', 'might', 'like'...","['rt', 'thought', 'like', 'comics', 'unpc', 'c...","['rt', 'thought', 'like', 'comic', 'unpc', 'co...",1
34651,"Here's some unoriginal thought, or in other wo...",here's some unoriginal thought or in other wor...,here's some unoriginal thought or in other wor...,"[""here's"", 'some', 'unoriginal', 'thought', 'o...","['unoriginal', 'thought', 'words', 'widely', '...","['unorigin', 'thought', 'word', 'wide', 'known...",1
34651,"Here's some unoriginal thought, or in other wo...",here's some unoriginal thought or in other wor...,NaN,"[""here's"", 'some', 'unoriginal', 'thought', 'o...","['unoriginal', 'thought', 'words', 'widely', '...","['unorigin', 'thought', 'word', 'wide', 'known...",1
61054,Quack quack.,quack quack,quack quack,"['quack', 'quack']","['quack', 'quack']","['quack', 'quack']",0
61054,Quack quack.,quack quack,NaN,"['quack', 'quack']","['quack', 'quack']","['quack', 'quack']",0
86760,Do you think placing a win cork up one's ass i...,do you think placing a win cork up one's ass i...,do you think placing a win cork up one's ass i...,"['do', 'you', 'think', 'placing', 'a', 'win', ...","['think', 'placing', 'win', 'cork', 'ass', 'ef...","['think', 'place', 'win', 'cork', 'ass', 'effe...",0
86760,Do you think placing a win cork up one's ass i...,do you think placing a win cork up one's ass i...,NaN,"['do', 'you', 'think', 'placing', 'a', 'win', ...","['think', 'placing', 'win', 'cork', 'ass', 'ef...","['think', 'place', 'win', 'cork', 'ass', 'effe...",0
189435,"""\n\n Policy references and wiki syntax \n\nMy...",policy references and wiki syntax my apologie...,policy references and wiki syntax my apologie...,"[' ', 'policy', 'references', 'and', 'wiki', '...","['policy', 'references', 'wiki', 'syntax', 'ap...","['polici', 'refer', 'wiki', 'syntax', 'apolog'...",0
189435,"""\n\n Policy references and wiki syntax \n\nMy...",policy references and wiki syntax my apologie...,NaN,"[' ', 'policy', 'references', 'and', 'wiki', '...","['policy', 'references', 'wiki', 'syntax', 'ap...","['polici', 'refer', 'wiki', 'syntax', 'apolog'...",0


# TF-IDF

## TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words='english'
                        ,ngram_range = (1,2)
                        ,max_df = 0.7
                        ,min_df = 3
                        ,max_features = 10000
                        )

## Transform

In [ ]:
## CLENAED DATA

# RAW
df_clean_raw_tfidf = tfidf.fit_transform(df_cleaned['tokens_raw'])
print('df_clean_raw_tfidf Done!!!')

# STOPWORD
df_clean_stopword_tfidf = tfidf.fit_transform(df_cleaned['tokens_stop'])
print('df_clean_stopword_tfidf Done!!!')

# STOPWORD & STEMMING
df_clean_stemming_tfidf = tfidf.fit_transform(df_cleaned['tokens_stems'])
print('df_clean_stemming_tfidf Done!!!')

df_clean_raw_tfidf Done!!!
df_clean_stopword_tfidf Done!!!
df_clean_stemming_tfidf Done!!!


In [ ]:
## TYPO DATA

# RAW
df_typo_raw_tfidf = tfidf.fit_transform(df_typo['tokens_raw'])
print('df_typo_raw_tfidf Done!!!')

# STOPWORD
df_typo_stopword_tfidf = tfidf.fit_transform(df_typo['tokens_stop'])
print('df_typo_stopword_tfidf Done!!!')

# STOPWORD & STEMMING
df_typo_stemming_tfidf = tfidf.fit_transform(df_typo['tokens_stems'])
print('df_typo_stemming_tfidf Done!!!')

df_typo_raw_tfidf Done!!!
df_typo_stopword_tfidf Done!!!
df_typo_stemming_tfidf Done!!!


##  TF-IDF DF

# Modeling

In [ ]:
import pandas as pd
import numpy as np

eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]
eval

,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums


In [ ]:
dataset_list = [df_clean_raw_tfidf
,df_clean_stopword_tfidf
,df_clean_stemming_tfidf
,df_typo_raw_tfidf
,df_typo_stopword_tfidf
,df_typo_stemming_tfidf]

dataset_name = ["Clean_Raw"
,"Clean_Stopword"
,"Clean_Stemming"
,"Typo_Raw"
,"Typo_Stopword"
,"Typo_Stemming"]

dataset = {na:li for na,li in zip(dataset_name,dataset_list)}

## K_Fold

In [ ]:
import ast
df_cleaned['tokens_raw'] = df_cleaned['tokens_raw'].apply(lambda x : ast.literal_eval(x))

In [ ]:
from sklearn.model_selection import KFold
# K_Fold Cross Validation
kf = KFold(n_splits=5,random_state=42)
K_FOLD_LIST = []
k=1
for train_index, test_index in kf.split(df_cleaned):
  K_FOLD_LIST.append([train_index, test_index,k,df_cleaned['tokens_raw'].iloc[test_index].apply(len).sum()])
  k+=1

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
print("train index\t\t\t\t\ttest index\t\t\t\t\tk   word nums")
print(K_FOLD_LIST[0][0],K_FOLD_LIST[0][1],"     ",K_FOLD_LIST[0][2],K_FOLD_LIST[0][3])
print(K_FOLD_LIST[1][0],K_FOLD_LIST[1][1],K_FOLD_LIST[1][2],K_FOLD_LIST[1][3])
print(K_FOLD_LIST[2][0],K_FOLD_LIST[2][1],K_FOLD_LIST[2][2],K_FOLD_LIST[2][3])
print(K_FOLD_LIST[3][0],K_FOLD_LIST[3][1],K_FOLD_LIST[3][2],K_FOLD_LIST[3][3])
print(K_FOLD_LIST[4][0],K_FOLD_LIST[4][1],K_FOLD_LIST[4][2],K_FOLD_LIST[4][3])

train index					test index					k   word nums
[ 64320  64321  64322 ... 321593 321594 321595] [    0     1     2 ... 64317 64318 64319]       1 1955231
[     0      1      2 ... 321593 321594 321595] [ 64320  64321  64322 ... 128636 128637 128638] 2 1939953
[     0      1      2 ... 321593 321594 321595] [128639 128640 128641 ... 192955 192956 192957] 3 1951304
[     0      1      2 ... 321593 321594 321595] [192958 192959 192960 ... 257274 257275 257276] 4 1941812
[     0      1      2 ... 257274 257275 257276] [257277 257278 257279 ... 321593 321594 321595] 5 1935521


## Logistic Classification



In [ ]:
import time
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [ ]:
from sklearn.linear_model import LogisticRegression

eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]
eval

LR = LogisticRegression()

for data_name in dataset_name:
  # Performance Base DataFrame
  LR_list_base = ['LogisticRegression']
  LR_list_base.append(data_name)

  # Load Data
  X = dataset[data_name]
  y = df_cleaned['class']

  # K_Fold Cross Validation
  for train_index, test_index, k, word_nums in K_FOLD_LIST:
    # Get Train, Test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit and Inference
    LR.fit(X_train,y_train)
    print(f"{data_name} - {k} Trained")
    fitted = LR.predict(X_train)
    fitted_proba = LR.predict_proba(X_train)
    start = time.time()
    pred = LR.predict(X_test)
    inference_time = time.time()-start
    pred_proba = LR.predict_proba(X_test)
    print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
    # Evaluate
    train_acc = accuracy_score(y_train,fitted)
    train_auc = roc_auc_score(y_train,fitted_proba[:,1])
    train_f1 = f1_score(y_train,fitted)
    test_acc = accuracy_score(y_test,pred)
    test_auc = roc_auc_score(y_test,pred_proba[:,1])
    test_f1 = f1_score(y_test,pred)
    print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")

    LR_list = LR_list_base.copy()
    LR_list.append(k)
    LR_list.append(train_acc)
    LR_list.append(train_auc)
    LR_list.append(train_f1)
    LR_list.append(test_acc)
    LR_list.append(test_auc)
    LR_list.append(test_f1)
    LR_list.append(inference_time)
    LR_list.append(word_nums)

    eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
eval

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Clean_Raw - 1 Trained
Clean_Raw - 1 Inferenced : 0.004323005676269531s	train ACC : 0.9280111631088792 test ACC : 0.924580223880597 test F1 : 0.7993381592554291


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Clean_Raw - 2 Trained
Clean_Raw - 2 Inferenced : 0.003996849060058594s	train ACC : 0.9278209867185951 test ACC : 0.9247345263452479 test F1 : 0.8011828001149945


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Typo_Stemming - 4 Trained
Typo_Stemming - 4 Inferenced : 0.003911018371582031s	train ACC : 0.9246609685280845 test ACC : 0.9205833424027114 test F1 : 0.7897076986414162


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Typo_Stemming - 5 Trained
Typo_Stemming - 5 Inferenced : 0.0040438175201416016s	train ACC : 0.924956369982548 test ACC : 0.9201324647460315 test F1 : 0.7882260790699591


,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums
0,LogisticRegression,Clean_Raw,1.0,0.928011,0.962799,0.810358,0.924580,0.952188,0.799338,0.004323,1955231.0
0,LogisticRegression,Clean_Raw,2.0,0.927821,0.962503,0.809421,0.924735,0.953691,0.801183,0.003997,1939953.0
0,LogisticRegression,Clean_Raw,3.0,0.928058,0.962937,0.810174,0.923460,0.952043,0.798047,0.004046,1951304.0
0,LogisticRegression,Clean_Raw,4.0,0.927852,0.962681,0.809932,0.923615,0.952920,0.797960,0.004014,1941812.0
0,LogisticRegression,Clean_Raw,5.0,0.928365,0.963037,0.811184,0.922947,0.951012,0.796167,0.003793,1935521.0
0,LogisticRegression,Clean_Stopword,1.0,0.928147,0.962901,0.810792,0.924627,0.952275,0.799487,0.003796,1955231.0
0,LogisticRegression,Clean_Stopword,2.0,0.928097,0.962638,0.810308,0.925232,0.953987,0.802643,0.003810,1939953.0
0,LogisticRegression,Clean_Stopword,3.0,0.928435,0.963075,0.811198,0.923786,0.952135,0.799115,0.003875,1951304.0
0,LogisticRegression,Clean_Stopword,4.0,0.928353,0.962830,0.811371,0.924237,0.953177,0.799770,0.003987,1941812.0
0,LogisticRegression,Clean_Stopword,5.0,0.928513,0.963231,0.811554,0.923615,0.951333,0.798325,0.003800,1935521.0


In [ ]:
eval.to_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/logistic_performance.csv")

## RandomForest Classifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
distributions = {'max_depth': [None,5,10,15,20],
              'n_estimators':[100,200],
              'min_samples_split':[2,4,8],
              'oob_score':[True,False]
              }
rf = RandomForestClassifier()
clf = RandomizedSearchCV(rf, distributions, n_iter=10, cv=5, random_state=42,n_jobs=-1, verbose=3)
X = dataset['Typo_Stemming']
y = df_cleaned['class']
search = clf.fit(X[:10000], y[:10000])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  50 | elapsed:    2.2s remaining:   19.6s
[Parallel(n_jobs=-1)]: Done  22 out of  50 | elapsed:    4.2s remaining:    5.4s
[Parallel(n_jobs=-1)]: Done  39 out of  50 | elapsed:    7.6s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   50.7s finished


In [ ]:
search.best_params_

{'max_depth': None,
 'min_samples_split': 4,
 'n_estimators': 100,
 'oob_score': False}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]


for data_name in dataset_name:
  
  LR = RandomForestClassifier(max_depth=None,min_samples_split=4,n_estimators=100,oob_score=False,n_jobs=-1,verbose=10)
  
  # Performance Base DataFrame
  LR_list_base = ['RandomForestClassifier']
  LR_list_base.append(data_name)

  # Load Data
  X = dataset[data_name]
  y = df_cleaned['class']

  # K_Fold Cross Validation
  for train_index, test_index, k, word_nums in K_FOLD_LIST:
    # Get Train, Test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit and Inference
    LR.fit(X_train,y_train)
    print(f"{data_name} - {k} Trained")
    fitted = LR.predict(X_train)
    fitted_proba = LR.predict_proba(X_train)
    start = time.time()
    pred = LR.predict(X_test)
    inference_time = time.time()-start
    pred_proba = LR.predict_proba(X_test)
    print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
    # Evaluate
    train_acc = accuracy_score(y_train,fitted)
    train_auc = roc_auc_score(y_train,fitted_proba[:,1])
    train_f1 = f1_score(y_train,fitted)
    test_acc = accuracy_score(y_test,pred)
    test_auc = roc_auc_score(y_test,pred_proba[:,1])
    test_f1 = f1_score(y_test,pred)
    print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")

    LR_list = LR_list_base.copy()
    LR_list.append(k)
    LR_list.append(train_acc)
    LR_list.append(train_auc)
    LR_list.append(train_f1)
    LR_list.append(test_acc)
    LR_list.append(test_auc)
    LR_list.append(test_f1)
    LR_list.append(inference_time)
    LR_list.append(word_nums)
    eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
eval

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100building tree 5 of 100
building tree 6 of 100

building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100building tree 15 of 100
building tree 16 of 100
building tree 17 of 100

building tree 18 of 100building tree 19 of 100
building tree 20 of 100

building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100building tree 31 of 100

building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100building tree 38 of 100

building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   33.7s


building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   34.9s


...

[Parallel(n_jobs=-1)]: Done  65 out of 100 | elapsed:  1.1min remaining:   35.2s
[Parallel(n_jobs=-1)]: Done  76 out of 100 | elapsed:  1.1min remaining:   20.9s
[Parallel(n_jobs=-1)]: Done  87 out of 100 | elapsed:  1.3min remaining:   11.9s
[Parallel(n_jobs=-1)]: Done  98 out of 100 | elapsed:  1.3min remaining:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.


Typo_Stemming - 5 Trained


[Parallel(n_jobs=40)]: Done   5 tasks      | elapsed:    1.0s
[Parallel(n_jobs=40)]: Done  18 tasks      | elapsed:    1.0s
[Parallel(n_jobs=40)]: Done  32 out of 100 | elapsed:    1.1s remaining:    2.3s
[Parallel(n_jobs=40)]: Done  43 out of 100 | elapsed:    1.9s remaining:    2.6s
[Parallel(n_jobs=40)]: Done  54 out of 100 | elapsed:    2.0s remaining:    1.7s
[Parallel(n_jobs=40)]: Done  65 out of 100 | elapsed:    2.1s remaining:    1.1s
[Parallel(n_jobs=40)]: Done  76 out of 100 | elapsed:    2.1s remaining:    0.7s
[Parallel(n_jobs=40)]: Done  87 out of 100 | elapsed:    2.8s remaining:    0.4s
[Parallel(n_jobs=40)]: Done  98 out of 100 | elapsed:    2.9s remaining:    0.1s
[Parallel(n_jobs=40)]: Done 100 out of 100 | elapsed:    2.9s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   5 tasks      | elapsed:    1.0s
[Parallel(n_jobs=40)]: Done  18 tasks      | elapsed:    1.0s
[Parallel(n_jobs=40)]: Done  32 

Typo_Stemming - 5 Inferenced : 0.8371508121490479s	train ACC : 0.9919192154759268 test ACC : 0.9223402105132231 test F1 : 0.8029663524121337


,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums
0,RandomForestClassifier,Clean_Raw,1.0,0.991231,0.998442,0.978948,0.925684,0.948125,0.811781,1.035970,1955231.0
0,RandomForestClassifier,Clean_Raw,2.0,0.991332,0.998444,0.979160,0.926585,0.948577,0.814664,0.940982,1939953.0
0,RandomForestClassifier,Clean_Raw,3.0,0.991445,0.998444,0.979427,0.925170,0.947200,0.811543,0.935964,1951304.0
0,RandomForestClassifier,Clean_Raw,4.0,0.991103,0.998442,0.978620,0.924937,0.946082,0.810637,1.035798,1941812.0
0,RandomForestClassifier,Clean_Raw,5.0,0.991647,0.998457,0.979934,0.924564,0.944884,0.809696,0.935395,1935521.0
0,RandomForestClassifier,Clean_Stopword,1.0,0.991002,0.998344,0.978385,0.925373,0.947870,0.810905,1.034342,1955231.0
0,RandomForestClassifier,Clean_Stopword,2.0,0.991060,0.998324,0.978499,0.926973,0.948607,0.815999,1.035531,1939953.0
0,RandomForestClassifier,Clean_Stopword,3.0,0.991169,0.998356,0.978749,0.925139,0.947419,0.811021,1.035096,1951304.0
0,RandomForestClassifier,Clean_Stopword,4.0,0.990874,0.998333,0.978061,0.926025,0.947355,0.813441,1.036735,1941812.0
0,RandomForestClassifier,Clean_Stopword,5.0,0.991126,0.998353,0.978665,0.924113,0.945227,0.808866,1.036548,1935521.0


In [ ]:
eval.to_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/randomforest_performance.csv")

In [ ]:
pd.concat([pd.read_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/randomforest_performance.csv"),pd.read_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/logistic_performance.csv")]).groupby(['Model','Dataset']).mean().sort_values('Test_Score(F1)',ascending=False)

Unnamed: 0  ...  words_nums
Model                  Dataset                     ...            
RandomForestClassifier Clean_Stemming           0  ...   1944764.2
                       Clean_Stopword           0  ...   1944764.2
                       Clean_Raw                0  ...   1944764.2
                       Typo_Stemming            0  ...   1944764.2
LogisticRegression     Clean_Stemming           0  ...   1944764.2
RandomForestClassifier Typo_Raw                 0  ...   1944764.2
                       Typo_Stopword            0  ...   1944764.2
LogisticRegression     Clean_Stopword           0  ...   1944764.2
                       Clean_Raw                0  ...   1944764.2
                       Typo_Stemming            0  ...   1944764.2
                       Typo_Stopword            0  ...   1944764.2
                       Typo_Raw                 0  ...   1944764.2

[12 rows x 10 columns]

## SVM

In [ ]:
from sklearn import svm
eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]


for data_name in dataset_name:
  
  LR = svm.SVC()
  
  # Performance Base DataFrame
  LR_list_base = ['svm']
  LR_list_base.append(data_name)

  # Load Data
  X = dataset[data_name]
  y = df_cleaned['class']

  # K_Fold Cross Validation
  for train_index, test_index, k, word_nums in K_FOLD_LIST:
    # Get Train, Test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit and Inference
    LR.fit(X_train,y_train)
    print(f"{data_name} - {k} Trained")
    fitted = LR.predict(X_train)
    fitted_proba = LR.predict_proba(X_train)
    start = time.time()
    pred = LR.predict(X_test)
    inference_time = time.time()-start
    pred_proba = LR.predict_proba(X_test)
    print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
    # Evaluate
    train_acc = accuracy_score(y_train,fitted)
    train_auc = roc_auc_score(y_train,fitted_proba[:,1])
    train_f1 = f1_score(y_train,fitted)
    test_acc = accuracy_score(y_test,pred)
    test_auc = roc_auc_score(y_test,pred_proba[:,1])
    test_f1 = f1_score(y_test,pred)
    print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")

    LR_list = LR_list_base.copy()
    LR_list.append(k)
    LR_list.append(train_acc)
    LR_list.append(train_auc)
    LR_list.append(train_f1)
    LR_list.append(test_acc)
    LR_list.append(test_auc)
    LR_list.append(test_f1)
    LR_list.append(inference_time)
    LR_list.append(word_nums)
    eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
eval

Clean_Raw - 1 Trained


In [ ]:
eval.to_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/svm_performance.csv")

In [ ]:
import pandas as pd
pd.read_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/lgbm_performance.csv")


,Unnamed: 0,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums
0,0,LGBMClassifier,Clean_Raw,1.0,0.926219,0.949357,0.804043,0.924160,0.944846,0.797156,0.031773,1955231.0
1,0,LGBMClassifier,Clean_Raw,2.0,0.926262,0.949226,0.803980,0.925030,0.945861,0.801105,0.033280,1939953.0
2,0,LGBMClassifier,Clean_Raw,3.0,0.926562,0.949781,0.804758,0.923397,0.942915,0.796867,0.031659,1951304.0
3,0,LGBMClassifier,Clean_Raw,4.0,0.926169,0.949133,0.803671,0.923755,0.943736,0.796919,0.033559,1941812.0
4,0,LGBMClassifier,Clean_Raw,5.0,0.926775,0.949640,0.805745,0.923786,0.943120,0.797872,0.032745,1935521.0
5,0,LGBMClassifier,Clean_Stopword,1.0,0.926134,0.949652,0.803856,0.923896,0.944805,0.796373,0.033673,1955231.0
6,0,LGBMClassifier,Clean_Stopword,2.0,0.926041,0.948981,0.803324,0.924952,0.945333,0.800792,0.033997,1939953.0
7,0,LGBMClassifier,Clean_Stopword,3.0,0.926395,0.949850,0.804153,0.923289,0.942375,0.796553,0.045657,1951304.0
8,0,LGBMClassifier,Clean_Stopword,4.0,0.926328,0.949284,0.804320,0.924066,0.944390,0.797848,0.032788,1941812.0
9,0,LGBMClassifier,Clean_Stopword,5.0,0.926418,0.949531,0.804688,0.923475,0.943356,0.797265,0.041351,1935521.0


## LGBM

In [ ]:
from lightgbm import LGBMClassifier

eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]


for data_name in dataset_name:
  
  LR = LGBMClassifier()
  
  # Performance Base DataFrame
  LR_list_base = ['LGBMClassifier']
  LR_list_base.append(data_name)

  # Load Data
  X = dataset[data_name]
  y = df_cleaned['class']

  # K_Fold Cross Validation
  for train_index, test_index, k, word_nums in K_FOLD_LIST:
    # Get Train, Test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit and Inference
    LR.fit(X_train,y_train)
    print(f"{data_name} - {k} Trained")
    fitted = LR.predict(X_train)
    fitted_proba = LR.predict_proba(X_train)
    start = time.time()
    pred = LR.predict(X_test)
    inference_time = time.time()-start
    pred_proba = LR.predict_proba(X_test)
    print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
    # Evaluate
    train_acc = accuracy_score(y_train,fitted)
    train_auc = roc_auc_score(y_train,fitted_proba[:,1])
    train_f1 = f1_score(y_train,fitted)
    test_acc = accuracy_score(y_test,pred)
    test_auc = roc_auc_score(y_test,pred_proba[:,1])
    test_f1 = f1_score(y_test,pred)
    print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")

    LR_list = LR_list_base.copy()
    LR_list.append(k)
    LR_list.append(train_acc)
    LR_list.append(train_auc)
    LR_list.append(train_f1)
    LR_list.append(test_acc)
    LR_list.append(test_auc)
    LR_list.append(test_f1)
    LR_list.append(inference_time)
    LR_list.append(word_nums)
    eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
eval

Clean_Raw - 1 Trained
Clean_Raw - 1 Inferenced : 0.03177309036254883s	train ACC : 0.9262193131112113 test ACC : 0.924160447761194 test F1 : 0.7971556886227545
Clean_Raw - 2 Trained
Clean_Raw - 2 Inferenced : 0.0332796573638916s	train ACC : 0.926262355360176 test ACC : 0.9250299289479003 test F1 : 0.8011054281471705
Clean_Raw - 3 Trained
Clean_Raw - 3 Inferenced : 0.03165912628173828s	train ACC : 0.9265616436758825 test ACC : 0.9233974408806107 test F1 : 0.796866625438054
Clean_Raw - 4 Trained
Clean_Raw - 4 Inferenced : 0.03355908393859863s	train ACC : 0.9261690706903454 test ACC : 0.9237550335048741 test F1 : 0.7969189995030644
Clean_Raw - 5 Trained
Clean_Raw - 5 Inferenced : 0.0327448844909668s	train ACC : 0.9267754210442442 test ACC : 0.9237861285156796 test F1 : 0.7978723404255319
Clean_Stopword - 1 Trained
Clean_Stopword - 1 Inferenced : 0.03367304801940918s	train ACC : 0.9261338018314962 test ACC : 0.923896144278607 test F1 : 0.7963725612546279
Clean_Stopword - 2 Trained
Clean_Sto

,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums
0,LGBMClassifier,Clean_Raw,1.0,0.926219,0.949357,0.804043,0.924160,0.944846,0.797156,0.031773,1955231.0
0,LGBMClassifier,Clean_Raw,2.0,0.926262,0.949226,0.803980,0.925030,0.945861,0.801105,0.033280,1939953.0
0,LGBMClassifier,Clean_Raw,3.0,0.926562,0.949781,0.804758,0.923397,0.942915,0.796867,0.031659,1951304.0
0,LGBMClassifier,Clean_Raw,4.0,0.926169,0.949133,0.803671,0.923755,0.943736,0.796919,0.033559,1941812.0
0,LGBMClassifier,Clean_Raw,5.0,0.926775,0.949640,0.805745,0.923786,0.943120,0.797872,0.032745,1935521.0
0,LGBMClassifier,Clean_Stopword,1.0,0.926134,0.949652,0.803856,0.923896,0.944805,0.796373,0.033673,1955231.0
0,LGBMClassifier,Clean_Stopword,2.0,0.926041,0.948981,0.803324,0.924952,0.945333,0.800792,0.033997,1939953.0
0,LGBMClassifier,Clean_Stopword,3.0,0.926395,0.949850,0.804153,0.923289,0.942375,0.796553,0.045657,1951304.0
0,LGBMClassifier,Clean_Stopword,4.0,0.926328,0.949284,0.804320,0.924066,0.944390,0.797848,0.032788,1941812.0
0,LGBMClassifier,Clean_Stopword,5.0,0.926418,0.949531,0.804688,0.923475,0.943356,0.797265,0.041351,1935521.0


In [ ]:
eval.to_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/lgbm_performance.csv")

## LSTM

In [ ]:
import tensorflow as tf

for data_name in dataset_name:

  # Performance Base DataFrame
  LR_list_base = ['BiLSTM']
  LR_list_base.append(data_name)

  # Load Data
  X = dataset[data_name]
  y = df_cleaned['class']

  # K_Fold Cross Validation
  for train_index, test_index, k, word_nums in K_FOLD_LIST:
    # Get Train, Test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train = X_train.toarray().reshape(X_train.shape[0],1,X_train.shape[-1])
    X_test = X_test.toarray().reshape(X_test.shape[0],1,X_test.shape[-1])

    LR = tf.keras.Sequential([
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(1)
      ])

    LR.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(1e-4),
                metrics=['accuracy'])

    # Fit and Inference
    LR.fit(X_train,y_train,epochs=20,batchsize=2)
    print(f"{data_name} - {k} Trained")
    fitted = LR.predict(X_train)
    fitted_proba = LR.predict_proba(X_train)
    start = time.time()
    pred = LR.predict(X_test)
    inference_time = time.time()-start
    pred_proba = LR.predict_proba(X_test)
    print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
    # Evaluate
    train_acc = accuracy_score(y_train,fitted)
    train_auc = roc_auc_score(y_train,fitted_proba[:,1])
    test_acc = accuracy_score(y_test,pred)
    test_auc = roc_auc_score(y_test,pred_proba[:,1])
    print(f"train ACC : {train_acc} test ACC : {test_acc}")

    LR_list = LR_list_base.copy()
    LR_list.append(k)
    LR_list.append(train_acc)
    LR_list.append(train_auc)
    LR_list.append(test_acc)
    LR_list.append(test_auc)
    LR_list.append(inference_time)
    LR_list.append(word_nums)

    eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
eval